In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.3 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

# df = pd.read_csv('/content/drive/MyDrive/training.csv')
# df.head()
df_train = pd.read_csv('/content/training.csv', sep =',')

print(df_train.head())

                                               Input Sentiment
0          I can't understand the method of teaching  Negative
1  The instructor was interested in the students ...  Positive
2  The instructor don't use any examples for expl...  Negative
3          Teaching is good but always late to class   Neutral
4                               Explaination is poor  Negative


In [ ]:
df_train['Sentiment'].value_counts()

Positive    576
Negative    493
Neutral     233
Name: Sentiment, dtype: int64

In [ ]:
df_train['Sentiment'].isnull().values.any() #checking for anu null values

False

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
encoded_dict = {'Neutral':0,'Positive':1, 'Negative':2 }
df_train['Sentiment'] = df_train.Sentiment.map(encoded_dict)

In [ ]:
df_train

,Input,Sentiment
0,I can't understand the method of teaching,2
1,The instructor was interested in the students ...,1
2,The instructor don't use any examples for expl...,2
3,Teaching is good but always late to class,0
4,Explaination is poor,2
...,...,...
1297,All is there except indoor games,0
1298,All is well except fitness equipments,0
1299,Everything is fine except ground condition,0
1300,Ground is in best condition but no gaming equi...,0


In [ ]:
y_train = to_categorical(df_train.Sentiment)

In [ ]:
y_train

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
x_train = tokenizer(
    text=df_train.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
x_train['input_ids']

<tf.Tensor: shape=(1302, 24), dtype=int32, numpy=
array([[  101,   146,  1169, ...,     0,     0,     0],
       [  101,  1109, 10332, ...,     0,     0,     0],
       [  101,  1109, 10332, ...,     0,     0,     0],
       ...,
       [  101,  5268,  1110, ...,     0,     0,     0],
       [  101,  8149,  1110, ...,     0,     0,     0],
       [  101,  1302,  4778, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 24
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 24)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 24)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 24,                                            

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,y = y_train,epochs=15,batch_size=36)

Epoch 1/15


37/37 [==============================] - 64s 262ms/step - loss: 0.1836 - balanced_accuracy: 0.9462
Epoch 2/15
37/37 [==============================] - 9s 231ms/step - loss: 0.1273 - balanced_accuracy: 0.9570
Epoch 3/15
37/37 [==============================] - 9s 235ms/step - loss: 0.1173 - balanced_accuracy: 0.9531
Epoch 4/15
37/37 [==============================] - 9s 234ms/step - loss: 0.0940 - balanced_accuracy: 0.9662
Epoch 5/15
37/37 [==============================] - 8s 220ms/step - loss: 0.0803 - balanced_accuracy: 0.9724
Epoch 6/15
37/37 [==============================] - 8s 230ms/step - loss: 0.0790 - balanced_accuracy: 0.9693
Epoch 7/15
37/37 [==============================] - 8s 215ms/step - loss: 0.0713 - balanced_accuracy: 0.9700
Epoch 8/15
37/37 [==============================] - 8s 217ms/step - loss: 0.0966 - balanced_accuracy: 0.9639
Epoch 9/15
37/37 [==============================] - 8s 220ms/step - loss: 0.0733 - balanced_accuracy: 0.9700
Epoch 10/15
37/37 [==========

In [ ]:
texts = input(str('input the text:'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=24,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the text:superb explanation by professor
1/1 [==============================] - 0s 46ms/step
Neutral 0.05916176
Positive 99.93751
Negative 0.0033405821
